<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [11]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [21]:
def model_trainer(neighbours):
    neighbours=int(neighbours)
    print(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]


In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    metric = model_trainer(neighbours)
    print([neighbours,metric[0],metric[1],metric[2]],',')
    return {'loss': -metric[0], 'status': STATUS_OK }

space={'neighbours': hp.uniform('neighbours', 1, 50)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

8                                                                                                                      
  0%|                                                                            | 0/100 [00:00<?, ?it/s, best loss: ?]

D:\Programas\Anaconda\Anaconda\Lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide



[8.239710799257978, 0.05731716329029847, 0.039215270413573734, 0.16671394741772194]                                    
,                                                                                                                      
31                                                                                                                     
[31.748273995731754, 0.06022051597296333, 0.04118769883351015, 0.17246361896323287]                                    
,                                                                                                                      
40                                                                                                                     
[40.43299652457653, 0.05928796587702535, 0.04060445387062568, 0.16944668187862819]                                     
,                                                                                                                      
18                                      

29                                                                                                                     
[29.483415156068123, 0.05943516315027853, 0.04065747613997883, 0.17272767166318043]                                    
,                                                                                                                      
38                                                                                                                     
[38.29912684244958, 0.06025328174154451, 0.04129374337221646, 0.17410502703857203]                                     
,                                                                                                                      
29                                                                                                                     
[29.252130776274548, 0.06158308741340364, 0.04212089077412514, 0.17804163049986918]                                    
,                                       

,                                                                                                                      
35                                                                                                                     
[35.958141359904594, 0.0584139521297219, 0.040095440084835655, 0.16913557772729093]                                    
,                                                                                                                      
42                                                                                                                     
[42.926255301739054, 0.05882921247010585, 0.04023329798515377, 0.1702203546435205]                                     
,                                                                                                                      
39                                                                                                                     
[39.46345784754656, 0.05944853244780545,

[11.183237140603179, 0.059156046471473364, 0.04042417815482505, 0.1741271350001904]                                    
,                                                                                                                      
27                                                                                                                     
[27.630863741853492, 0.0584191653922818, 0.039978791092258784, 0.1687719527249841]                                     
,                                                                                                                      
18                                                                                                                     
[18.528166693186883, 0.059151059051600445, 0.040455991516437, 0.17173380363294038]                                     
,                                                                                                                      
34                                      

22                                                                                                                     
[22.37474757138822, 0.05812476982709031, 0.03983032873807003, 0.1667149229687202]                                      
,                                                                                                                      
37                                                                                                                     
[37.24425965418398, 0.05898465020614523, 0.04039236479321323, 0.16989000094860982]                                     
,                                                                                                                      
41                                                                                                                     
[41.96942674458991, 0.060525809892845686, 0.04136797454931071, 0.17532283092996032]                                    
,                                       

In [26]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','F1','Precision','Recall']
metric.head(5)

,neighbours,F1,Precision,Recall
0,8.239711,0.057317,0.039215,0.166714
1,31.748274,0.060221,0.041188,0.172464
2,40.432997,0.059288,0.040604,0.169447
3,18.379104,0.057816,0.039385,0.167235
4,41.363564,0.061231,0.041919,0.174962


In [29]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,neighbours,F1,Precision,Recall
25,29.252131,0.061583,0.042121,0.178042
4,41.363564,0.061231,0.041919,0.174962
15,26.858744,0.060764,0.041591,0.175332
77,20.163610,0.060589,0.041389,0.174556
51,33.959412,0.060583,0.041400,0.176024
93,41.969427,0.060526,0.041368,0.175323
72,23.930736,0.060449,0.041251,0.175460
14,20.351559,0.060420,0.041262,0.175858
54,24.603637,0.060365,0.041347,0.175259
66,30.507329,0.060358,0.041347,0.176060


In [28]:
metric.to_csv('metric_F1.csv', index=False)